In [1]:
using LinearAlgebra
using ForwardDiff

In [2]:
# Compute Jacobian using finite differences
function finite_diff_jacobian(f, x::Vector{Float64}, h::Float64 = 1e-6)
    n = length(x)
    J = zeros(n, n)
    fx = f(x)  # Compute f(x) once to avoid redundant evaluations
    for j in 1:n
        x_perturbed = copy(x)
        x_perturbed[j] += h
        J[:, j] = (f(x_perturbed) - fx) / h
    end
    return J
end

# Compute Jacobian using automatic differentiation
function autodiff_jacobian(f, x::Vector{Float64})
    return ForwardDiff.jacobian(f, x)
end

# Newton solver (already given) using finite differences
function Newton_finite_diff(f, x::Vector{Float64}; tol::Float64 = 1e-8, max_iter::Int64 = 1000)
    for i in 1:max_iter
        J = finite_diff_jacobian(f, x)
        δx = -J \ f(x)  # Newton update step
        x += δx
        if norm(δx) < tol
            return x, i, 0
        end
    end
    println("Failed to converge after $max_iter iterations, returning last computed solution.")
    return x, max_iter, 1
end

# Newton solver using automatic differentiation
function Newton_autodiff(f, x::Vector{Float64}; tol::Float64 = 1e-8, max_iter::Int64 = 1000)
    for i in 1:max_iter
        J = autodiff_jacobian(f, x)
        δx = -J \ f(x)  # Newton update step
        x += δx
        if norm(δx) < tol
            return x, i, 0
        end
    end
    println("Failed to converge after $max_iter iterations, returning last computed solution.")
    return x, max_iter, 1
end

Newton_autodiff (generic function with 1 method)

In [5]:
# Initial guess
x0 = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

# Define the nonlinear function
# function F(x::Vector)
#     return [10 * (x[1] - x[2]^2); 1 - x[1]]
# end

function F(x::Vector)
    return [
        x[1]^2 + x[2]^2 + x[3]^2 - 1,
        x[1] * exp(x[2]) - x[3],
        x[2] * cos(x[3]) + x[1] * x[3] - 0.5
    ]
end

function F10(x::Vector)
    return [
        x[1]^2 + x[2]^2 + x[3]^2 - 1,                    # Sphere constraint
        x[1] * exp(x[2]) - x[3] + sin(x[4]),             # Exponential + trig interaction
        x[2] * cos(x[3]) + x[1] * x[3] - 0.5,            # Trigonometric + polynomial
        x[4]^2 + x[5]^2 - x[6],                          # Quadratic constraint
        x[5] * exp(x[6]) - x[7]^2,                       # Exponential and quadratic
        x[6] * cos(x[7]) + x[5] * x[8] - 1,              # Trigonometric + interaction
        x[8]^2 + x[9]^2 + x[10]^2 - 2,                   # Sphere-like constraint
        x[7] * exp(x[8]) - x[9] + sin(x[10]),            # Another exp + trig interaction
        x[9] * cos(x[10]) + x[8] * x[10] - 0.3,          # Trigonometric + polynomial
        x[1] + x[2] + x[3] + x[4] + x[5] - 2             # Linear constraint across variables
    ]
end


# Solve using finite differences
@time solution_fd, iter_fd, status_fd = Newton_finite_diff(F10, x0)
println("Finite Differences: Solution = $solution_fd, Iterations = $iter_fd")

ArgumentError: ArgumentError: matrix contains Infs or NaNs

In [12]:
# Solve using automatic differentiation
@time solution_ad, iter_ad, status_ad = Newton_autodiff(F10, x0)
println("Automatic Differentiation: Solution = $solution_ad, Iterations = $iter_ad")

ArgumentError: ArgumentError: matrix contains Infs or NaNs

In [6]:
using ForwardDiff
using SpecialFunctions  # Import SpecialFunctions for Bessel functions

# Define your nonlinear system using Bessel functions
function F(x::Vector)
    return [
        besseli(0, x[1]) + x[2]^2 - 1,   # Bessel function J_0(x1)
        x[1] * besselk(0, x[2]) - x[3],  # Bessel function Y_1(x2)
        besseli(1, x[3]) + x[1] * x[2]   # Bessel function J_1(x3)
    ]
end

# Compute the Jacobian using ForwardDiff
function autodiff_jacobian(f, x::Vector{Float64})
    return ForwardDiff.jacobian(f, x)
end

# Test with an initial guess
x0 = [1.0, 2.0, 3.0]
J = autodiff_jacobian(F, x0)

println("Computed Jacobian: ", J)


Computed Jacobian: [0.565159103992485 4.0 0.0; 0.11389387274953341 -0.13986588181652246 -1.0; 2.0 1.0 3.563002513397487]


In [8]:
# check return type of F
typeof(F(x0))

Vector{Float64} (alias for Array{Float64, 1})